In [1]:
import warnings, requests
import pandas as pd
import numpy as np
import empyrical as ep
import plotly.express as px

from datetime import datetime

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import linear_kernel

warnings.filterwarnings('ignore')

c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\crist\.conda\envs\trading\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [8]:
data = pd.read_parquet('./funds_metrics.parquet')

In [9]:
data

,cnpj,return,avg_return,vol,max_dd
0,00068305000135,0.389372,0.068037,0.003425,-0.003986
0,00071477000168,0.329269,0.058625,0.002791,-0.000071
0,00073041000108,0.412743,0.071609,0.002711,-0.000731
0,00083181000167,0.776607,0.121910,0.209643,-0.419639
0,00089915000115,0.392057,0.068450,0.009317,-0.003807
...,...,...,...,...,...
0,34246364000178,0.018718,0.004900,0.196084,-0.298122
0,34246372000114,-0.016945,-0.004481,0.136304,-0.218643
0,34246376000100,0.107445,0.026399,0.132033,-0.186718
0,34246380000160,0.374984,0.086901,0.003476,-0.006719


In [17]:
funds_series = pd.read_parquet(f'{tools.path}/funds_series.parquet')

In [16]:
inf_cadastral = pd.read_parquet(f'{tools.path}/inf_cadastral.parquet')
inf_cadastral = inf_cadastral[inf_cadastral.index.isin(data['cnpj'])]
inf_cadastral.to_parquet('./inf_cadastral.parquet')

In [19]:
funds_series = pd.read_parquet(f'{tools.path}/funds_series.parquet')
funds_series = funds_series[funds_series['CNPJ_FUNDO'].isin(data['cnpj'])]
funds_series.to_parquet('./funds_series.parquet')

In [20]:
inf_diario = pd.read_parquet(f'{tools.path}/inf_diario_all.parquet').reset_index()
inf_diario = inf_diario[inf_diario['CNPJ_FUNDO'].isin(data['cnpj'])]
inf_diario.to_parquet('./inf_diario_all.parquet')

In [2]:
class Tools:
    def __init__(self):
        self.path = 'D:/Disco/Data/CVM'

    def compute_metrics(self, fund_cnpj, funds_series, length):
        fund_data = funds_series[funds_series['CNPJ_FUNDO'] == fund_cnpj][['DT_COMPTC', 'VL_QUOTA']].tail(length)
        fund_data = fund_data.rename(columns={'DT_COMPTC':'date', 'VL_QUOTA':'value'})
        fund_data.drop_duplicates(keep='last', inplace=True)
        fund_data.set_index('date', inplace=True)

        fund_data['returns'] = ep.simple_returns(fund_data['value'])
        fund_data['returns'] = fund_data['returns'].replace([np.inf, -np.inf], np.nan)
        fund_data = fund_data.dropna()

        data = {
            'cnpj':fund_cnpj,
            'return': (1 + fund_data['returns']).cumprod()[-1] -1, 
            'avg_return': ep.annual_return(fund_data['returns']),
            'vol': np.std(fund_data['returns']) * np.sqrt(252),
            'max_dd': ep.max_drawdown(fund_data['returns'])
        }

        return pd.DataFrame([data])

    def getBCBSerie(self, serie_id, serie_name):
        data = requests.get(f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie_id} \
                            /dados?dataInicial=01-01-1970 \
                            &dataFinal={datetime.today().strftime('%d-%m-%Y')}").json()

        data = pd.DataFrame(data, dtype=float, columns=['data', 'valor']).rename(columns={'data':'date', 'valor':'value'})
        data.set_index('date', inplace=True)
        data['index_id'] = serie_id
        data['index_name'] = serie_name
        data.index = pd.to_datetime(data.index, format="%d/%m/%Y")
        
        return data
    
    def computeReturn(self, data, start_date, end_date):
        returns = data.loc[(data.index >= start_date) & (data.index <= end_date)]
        if(len(returns)>0):
            returns['value'] = (1 + returns['value'] / 100)
            returns.loc[returns.index.min()]['value'] = 1
            returns['value'] = (returns['value']).cumprod()
            returns['value'] = (returns['value'] - 1)
        else:
            returns = pd.DataFrame(pd.date_range(start_date, end_date), columns={'date'}).set_index('date')
            returns['value'] = 0

        return returns

In [3]:
tools = Tools()

In [37]:
selic = tools.getBCBSerie(11, 'SELIC')
selic.tail(5)

,value,index_id,index_name
date,,,
2024-06-10,0.03927,11,SELIC
2024-06-11,0.03927,11,SELIC
2024-06-12,0.03927,11,SELIC
2024-06-13,0.03927,11,SELIC
2024-06-14,0.03927,11,SELIC


In [22]:
inf_cadastral = pd.read_parquet(f'{tools.path}/inf_cadastral.parquet')
inf_cadastral = inf_cadastral[(inf_cadastral['FUNDO_EXCLUSIVO'] == "N") & (inf_cadastral['SIT'] == "EM FUNCIONAMENTO NORMAL")]
inf_cadastral.index = inf_cadastral.index.str.replace('.','').str.replace('/', '').str.replace('-', '')
print(len(inf_cadastral))

inf_cadastral.head(3)

19226


,TP_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,...,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR,INVEST_CEMPR_EXTER,CLASSE_ANBIMA
CNPJ_FUNDO,,,,,,,,,,,,,,,,,,,,,
00068305000135,FI,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2004-12-22,1994-05-26,744.0,None,EM FUNCIONAMENTO NORMAL,2004-12-02,1994-05-26,2023-07-01,...,42.040.639/0001-40,CAIXA DISTRIBUIDORA DE TÍTULOS E VALORES MOBIL...,49.928.567/0001-11,DELOITTE TOUCHE TOHMATSU AUDITORES INDEPENDENT...,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL,00.360.305/0001-04,CAIXA ECONOMICA FEDERAL,N,Renda Fixa Duração Livre Grau de Invest.
00071477000168,FI,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2005-03-29,1994-05-19,40584.0,None,EM FUNCIONAMENTO NORMAL,2005-03-24,1994-05-19,2024-04-01,...,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,00.000.000/0001-91,BANCO DO BRASIL S.A.,00.000.000/0001-91,BANCO DO BRASIL S.A.,N,Renda Fixa Simples
00073041000108,FI,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2005-07-28,1995-09-19,60674.0,None,EM FUNCIONAMENTO NORMAL,2005-07-20,1995-09-19,2024-04-01,...,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,00.000.000/0001-91,BANCO DO BRASIL S.A.,00.000.000/0001-91,BANCO DO BRASIL S.A.,N,Renda Fixa Duração Livre Crédito Livre


In [23]:
inf_diario = pd.read_parquet(f'{tools.path}/inf_diario_all.parquet').reset_index()
print(len(inf_diario))

#filtra as series conforme os cnpj validos
inf_diario = inf_diario[inf_diario['CNPJ_FUNDO'].isin(inf_cadastral.index.values)]
inf_diario['DT_COMPTC'] = pd.to_datetime(inf_diario['DT_COMPTC'], format='%Y-%m-%d')
print(len(inf_diario))

inf_diario.head(5)

65028183
30466856


,DT_COMPTC,CNPJ_FUNDO,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO
4,2005-01-03,03047880000140,4.355492e+06,24.517015,4.351392e+06,0.00,0.00,1,None
29,2005-01-03,03004427000156,1.149018e+08,3202.103206,1.149038e+08,0.00,0.00,2,None
33,2005-01-03,03009309000130,9.573164e+06,275.472374,9.571516e+06,0.00,0.00,2,None
39,2005-01-03,03029216000178,2.337527e+08,27.745054,2.847102e+08,991774.56,7561471.81,61,None
41,2005-01-03,05832360000173,8.862332e+07,1.152519,8.861256e+07,61751.87,10776.82,10,None


In [24]:
funds_series = pd.merge(inf_cadastral[['TP_FUNDO', 'DENOM_SOCIAL', 'CLASSE_ANBIMA', 'CLASSE', 'RENTAB_FUNDO', 'PUBLICO_ALVO']], 
         inf_diario[['DT_COMPTC', 'CNPJ_FUNDO', 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ']], 
         how='inner',left_on='CNPJ_FUNDO', right_on='CNPJ_FUNDO')

del inf_cadastral, inf_diario

#salva em arquivo os dados
funds_series.to_parquet(f'{tools.path}/funds_series.parquet')


In [25]:
funds_series.head(4)

,CNPJ_FUNDO,TP_FUNDO,DENOM_SOCIAL,CLASSE_ANBIMA,CLASSE,RENTAB_FUNDO,PUBLICO_ALVO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ
0,00068305000135,FI,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,Renda Fixa Duração Livre Grau de Invest.,Fundo de Renda Fixa,None,Público Geral,2005-01-03,90065264.83,7.825542,89939819.22
1,00068305000135,FI,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,Renda Fixa Duração Livre Grau de Invest.,Fundo de Renda Fixa,None,Público Geral,2005-01-04,90170464.25,7.830530,90086975.31
2,00068305000135,FI,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,Renda Fixa Duração Livre Grau de Invest.,Fundo de Renda Fixa,None,Público Geral,2005-01-05,90045393.65,7.834892,89950814.38
3,00068305000135,FI,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,Renda Fixa Duração Livre Grau de Invest.,Fundo de Renda Fixa,None,Público Geral,2005-01-06,90121083.85,7.839294,90015279.34


In [26]:

fund_data = funds_series[funds_series['CNPJ_FUNDO'] == '00068305000135'][['DT_COMPTC', 'VL_QUOTA']]
fund_data = fund_data.rename(columns={'DT_COMPTC':'date', 'VL_QUOTA':'value'})
fund_data.set_index('date', inplace=True)


#selic_temp = selic[selic.index.isin(fund_data.index.values)]


In [38]:
px.line(fund_data)

In [29]:
fund_data['returns'] = ep.simple_returns(fund_data['value'])
fund_data['returns'].replace([np.inf, -np.inf], np.nan)
fund_data = fund_data.dropna(subset=['returns'])

#selic['returns'] = ep.simple_returns(selic['value'])

#selic_temp = selic[selic.index.isin(fund_data.index.values)]


In [39]:
px.line(fund_data['returns'])

In [32]:
#testa o calculo das metricas
fund_cnpj = '00346750000110'
length = 1260
serie = funds_series[funds_series['CNPJ_FUNDO'] == fund_cnpj]
metrics = tools.compute_metrics(fund_cnpj, serie,length)

metrics

,cnpj,return,avg_return,vol,max_dd
0,00346750000110,0.28738,0.051862,0.039531,-0.058573


In [5]:
serie_size = 1260
funds_metrics = pd.DataFrame()

count=0
for cnpj in funds_series['CNPJ_FUNDO'].unique():
    serie = funds_series[funds_series['CNPJ_FUNDO'] == cnpj]
    
    if len(serie) > serie_size:
        print(f'computing metrics for: {cnpj}')
        metrics = tools.compute_metrics(cnpj, serie, serie_size)
        funds_metrics = pd.concat([funds_metrics, metrics])
        count +=1
        if count > 20:
            print('saving file...')
            count = 0
            funds_metrics.to_parquet('./funds_metrics.parquet')

funds_metrics.reset_index(inplace=True)
funds_metrics.to_parquet('./funds_metrics.parquet')

computing metrics for: 00068305000135
computing metrics for: 00071477000168
computing metrics for: 00073041000108
computing metrics for: 00083181000167
computing metrics for: 00089915000115
computing metrics for: 00102322000141
computing metrics for: 00180995000110
computing metrics for: 00185259000154
computing metrics for: 00194256000187
computing metrics for: 00211294000109
computing metrics for: 00222725000124
computing metrics for: 00222816000160
computing metrics for: 00280302000160
computing metrics for: 00306278000191
computing metrics for: 00322699000106
computing metrics for: 00346750000110
computing metrics for: 00360293000118
computing metrics for: 00398561000190
computing metrics for: 00400490000113
computing metrics for: 00463569000193
computing metrics for: 00524617000106
saving file...
computing metrics for: 00539553000117
computing metrics for: 00575922000127
computing metrics for: 00598452000117
computing metrics for: 00601692000123
computing metrics for: 007430260001

KeyboardInterrupt: 

In [34]:
data = pd.read_parquet('./funds_metrics.parquet')

data = data[data['vol'] <=1]
data['class'] =  pd.cut(data['vol'], bins=[0, 0.01, 0.02, 0.08, 0.10, data['vol'].max()], labels=[1,2,3,4,5])

data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

print(len(data))
data.head(6)

8208


,cnpj,return,avg_return,vol,max_dd,class
0,00068305000135,0.389372,0.068037,0.003425,-0.003986,1
1,00071477000168,0.329269,0.058625,0.002791,-0.000071,1
2,00073041000108,0.412743,0.071609,0.002711,-0.000731,1
3,00083181000167,0.776607,0.121910,0.209643,-0.419639,5
4,00089915000115,0.392057,0.068450,0.009317,-0.003807,1
5,00102322000141,-0.096708,-0.020152,0.347817,-0.758515,5


In [40]:
px.scatter(data, x='return', y='vol', color='class', width=1000)

In [41]:
px.bar(data, x='class', width=1000)

In [42]:
columns = ['return', 'avg_return', 'vol', 'max_dd', 'class']
cosine_similarities = linear_kernel(data[columns],data[columns])
indices = pd.Series(data.index, index=data['cnpj']).drop_duplicates()

#busca uma lista de fundos utilizando a similaridade do cosseno
def getFundsList(cnpj, size):
    idx = indices[cnpj]

    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_similar = sim_scores[1:size + 1]
    cnpj_indices = [i[0] for i in top_similar]
    cnpj_indices = [data.iloc[i]['cnpj'] for i in cnpj_indices]

    return cnpj_indices

#busca fundos do mesmo risco do usuario
def getFundsNewUser(user_risk, size):
    return data[data['class'] == user_risk].sample(n=size)

#### 1-lista de recomendações para usuário antigo

In [47]:
#risco inicial em 1
risks = [0, 0.01, 0.02, 0.08, 0.10]
users = pd.DataFrame()

id = 1
user_risk = 1
funds_list = data[(data['vol'] >= risks[user_risk-1]) & (data['vol'] < risks[user_risk])].sample(n=5)
portfolio_risk = funds_list['vol'].mean()

user = {
    'userid': id,
    'user_risk': user_risk,
    'portfolio_risk': portfolio_risk,
    'funds_list': funds_list
}

user

users = pd.concat([users, pd.DataFrame([user])])
user['funds_list']

,cnpj,return,avg_return,vol,max_dd,class
551,04150673000189,0.453582,0.077739,0.002897,-0.002733,1
3735,16565016000181,0.460748,0.078801,0.007934,-0.016253,1
3162,13442691000125,0.235488,0.043235,0.003926,-0.015013,1
6016,26253895000160,0.467642,0.079818,0.002791,-0.002005,1
6239,26695435000191,0.454457,0.077869,0.002830,-0.002912,1


In [48]:
#recomenda uma lista para usuario antigo
cnpj_user = user['funds_list'].sample(n=1)['cnpj'].values[0]
rec_user = getFundsList(cnpj_user, 5)
data[data['cnpj'].isin(rec_user)]

,cnpj,return,avg_return,vol,max_dd,class
1714,08843959000109,12.501519,0.683667,0.322012,-0.267813,5
4629,19599474000120,10.863123,0.640629,0.239518,-0.132979,5
5342,22975153000197,26.305992,0.938562,0.342722,-0.200510,5
5374,23129095000143,12.663850,0.687699,0.586140,-0.725603,5
7888,31933729000144,11.256633,0.651380,0.648862,-0.261140,5


#### 2 -lista de recomendações para usuário novo

In [49]:
#recomendacao para novo usuario
id = 2
user_risk = 3
funds_list = getFundsNewUser(user_risk, 5)
portfolio_risk = funds_list['vol'].mean()

user = {
    'userid': id,
    'user_risk': user_risk,
    'portfolio_risk': portfolio_risk,
    'funds_list': funds_list
}

user

users = pd.concat([users, pd.DataFrame([user])])

In [50]:
#recomendações
user['funds_list']

,cnpj,return,avg_return,vol,max_dd,class
5651,24164265000193,0.247250,0.045216,0.078411,-0.129418,3
5754,24592515000196,0.480335,0.081681,0.047813,-0.118623,3
5708,24405001000184,0.224616,0.041391,0.053189,-0.084006,3
1784,08973371000161,0.200493,0.037253,0.057482,-0.089402,3
5904,25382586000127,0.179620,0.033617,0.059730,-0.095792,3


#### 3-uma recomendação para usuário antigo

In [52]:
#recomenda item novo para usuario antigo
id = 3
user_risk = 4
funds_list = data[(data['vol'] >= risks[user_risk-1]) & (data['vol'] < risks[user_risk])].sample(n=5)
portfolio_risk = funds_list['vol'].mean()

cnpj_user = user['funds_list'].sample(n=1)['cnpj'].values[0]
rec_user = getFundsList(cnpj_user, 1)
data[data['cnpj'].isin(rec_user)]

,cnpj,return,avg_return,vol,max_dd,class
5342,22975153000197,26.305992,0.938562,0.342722,-0.20051,5
